<a href="https://colab.research.google.com/github/DaveMSU/Home-prices-prediction.-Kaggle-/blob/master/Home_prices_GB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Препарация train.csv и test.csv:**

In [0]:
import warnings
warnings.filterwarnings("ignore")

In [0]:
import numpy as np
import pandas as pd

train_df = pd.read_csv('Train.csv')
train_df['time'] = 0
train_df['time'] = pd.to_datetime(train_df['date']).astype(int)/ 10**10 
train_df = train_df.drop(['date'], axis=1)
test_df  = pd.read_csv('Test.csv')
test_df['time'] = 0
test_df['time'] = pd.to_datetime(test_df['date']).astype(int)/ 10**10
test_df = test_df.drop(['date'], axis=1)

train_df.head()

,id,street_id,build_tech,floor,area,rooms,balcon,metro_dist,g_lift,n_photos,kw1,kw2,kw3,kw4,kw5,kw6,kw7,kw8,kw9,kw10,kw11,kw12,kw13,price,time
0,0,531,NaN,12,35,1,1,25.0,NaN,1,0,1,0,0,0,0,0,0,0,0,0,0,0,3721000,129384000.0
1,1,385,2.0,15,46,1,2,15.0,1.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,7431000,129384000.0
2,2,51,0.0,1,53,3,0,30.0,0.0,7,0,1,0,0,0,0,0,0,0,0,0,0,0,3081000,129384000.0
3,3,76,0.0,1,60,3,0,30.0,0.0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,3847000,129384000.0
4,4,114,0.0,2,43,2,0,30.0,0.0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,2323000,129384000.0


In [0]:
train_df.g_lift.unique()

array([nan,  1.,  0.])

In [0]:
proc_test_df = test_df
query_build_tech = test_df['build_tech'].isnull()
proc_test_df.loc[query_build_tech, ['build_tech']] = test_df.loc[~query_build_tech, 'build_tech'].median()

query_metro_dist = test_df['metro_dist'].isnull()
proc_test_df.loc[query_metro_dist, ['metro_dist']] = test_df.loc[~query_metro_dist, 'metro_dist'].median()

query_g_lift = test_df['g_lift'].isnull()
proc_test_df.loc[query_g_lift, ['g_lift']] = test_df.loc[~query_g_lift, 'g_lift'].median()

#proc_test_df.isnull().sum(), proc_test_df.shape


proc_train_df = train_df
query_build_tech = train_df['build_tech'].isnull()
proc_train_df.loc[query_build_tech, ['build_tech']] = proc_train_df.loc[~query_build_tech, 'build_tech'].median()

query_metro_dist = train_df['metro_dist'].isnull()
proc_train_df.loc[query_metro_dist, ['metro_dist']] = proc_train_df.loc[~query_metro_dist, 'metro_dist'].median()

query_g_lift = train_df['g_lift'].isnull()
proc_train_df.loc[query_g_lift, ['g_lift']] = proc_train_df.loc[~query_g_lift, 'g_lift'].median()

#proc_train_df.isnull().sum(), proc_train_df.shape

In [0]:
proc_train_df.head(20)

In [0]:
X_data = proc_train_df.drop(['price', 'id'], axis = 1).astype("float64").values
y_data = proc_train_df['price'].astype("float64").values
X_test = proc_test_df.drop(['id'], axis = 1).astype("float64").values

print('X_train:\nshape: ', X_data.shape, '\ndtype: ', X_data.dtype)
print('y_train:\nshape: ', y_data.shape, '\ndtype: ', y_data.dtype)
print('X_test:\nshape: ', X_test.shape, '\ndtype: ', X_test.dtype)

X_train:
shape:  (100000, 23) 
dtype:  float64
y_train:
shape:  (100000,) 
dtype:  float64
X_test:
shape:  (100000, 23) 
dtype:  float64


In [0]:
#Библиотеки моделей для решения задачи регрессии.

from sklearn              import tree
from sklearn.linear_model import LinearRegression
from sklearn.neighbors    import KNeighborsRegressor
from sklearn.svm          import SVR
from sklearn.ensemble     import RandomForestRegressor
#main one under!
from sklearn.ensemble     import GradientBoostingRegressor

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics.scorer  import make_scorer
from sklearn.metrics         import mean_squared_error as MSE
from sklearn.metrics         import mean_absolute_error as MAE
from sklearn.metrics         import explained_variance_score as EVS


X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size = 0.333, random_state=7)

print('X_train.shape: ', X_train.shape)
print('y_train.shape: ', y_train.shape)
print('X_val.shape: ', X_val.shape)
print('y_val.shape: ', y_val.shape)

X_train.shape:  (66700, 23)
y_train.shape:  (66700,)
X_val.shape:  (33300, 23)
y_val.shape:  (33300,)


In [0]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics.scorer  import make_scorer

In [0]:
import time

#***Заполнение пропусков более умным способом:***

In [0]:
#Библиотеки моделей для решения задачи регрессии.

from sklearn.neighbors    import KNeighborsClassifier
from sklearn.ensemble     import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
#main one under!
from sklearn.ensemble     import GradientBoostingClassifier

In [0]:
#Метрики для классификации.

from sklearn.metrics import accuracy_score as ACC
from sklearn.metrics import f1_score       as F1
from sklearn.metrics import roc_auc_score  as ROC

#***'build_tech':***

In [0]:
label_name = 'build_tech'
X_pre_data = np.concatenate( (train_df.loc[ ~train_df[label_name].isnull() ]\
                              .drop(['metro_dist', 'g_lift', 'build_tech', 'price', 'id'], axis = 1)\
                              .astype("float64").values,\
                              test_df.loc[ ~test_df[label_name].isnull() ]\
                              .drop(['metro_dist', 'g_lift', 'build_tech', 'id'], axis = 1)\
                              .astype(int).values),\
                              axis=0)

                    
                  
y_pre_data = np.concatenate( (train_df.loc[ ~train_df[label_name]\
                                      .isnull() ][label_name].astype(int).values,
                              test_df.loc[ ~test_df[label_name]\
                                     .isnull() ][label_name].astype(int).values),\
                              axis=0)
                              

print('X_pre_train:\nshape: ', X_pre_data.shape, '\ndtype: ', X_pre_data.dtype)
print('y_pre_train:\nshape: ', y_pre_data.shape, '\ndtype: ', y_pre_data.dtype)

X_pre_train:
shape:  (140127, 20) 
dtype:  float64
y_pre_train:
shape:  (140127,) 
dtype:  int64


In [0]:
#begin:
t_before = time.time()

X_train, X_val, y_train, y_val = train_test_split(X_pre_data, y_pre_data, test_size = 0.333, random_state=7)

model = RandomForestClassifier(random_state=7)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)

acc_score = ACC(y_pred, y_val)
f1_score  = F1(y_pred, y_val, average='weighted')

model.fit(X_pre_data, y_pre_data)

t_after = time.time()
#end.

print('acc_score:', "%.3f" % acc_score, '  ',\
      'f1_score:',  "%.3f" % f1_score, '  ',\
      'time:', "%.2f" % (t_after - t_before))

acc_score: 0.930    f1_score: 0.930    time: 2.38


In [0]:
X_pre_test_train = train_df.loc[ train_df[label_name].isnull() ].drop(['metro_dist', 'g_lift', 'build_tech', 'price', 'id'], axis=1).astype("float64").values
y_pre_test_train = model.predict(X_pre_test_train)

In [0]:
train_df.loc[ train_df[label_name].isnull(), label_name ] = y_pre_test_train

In [0]:
X_pre_test = test_df.loc[ test_df[label_name].isnull() ].drop(['metro_dist', 'g_lift', 'build_tech', 'id'], axis=1).astype("float64").values
y_pre_test = model.predict(X_pre_test)
test_df.loc[ test_df[label_name].isnull(), label_name ] = y_pre_test

#***'g_lift':***


In [0]:
label_name = 'g_lift'
X_pre_data = np.concatenate( (train_df.loc[ ~train_df[label_name].isnull() ]\
                              .drop(['metro_dist', 'g_lift', 'build_tech', 'price', 'id'], axis = 1)\
                              .astype("float64").values,\
                              test_df.loc[ ~test_df[label_name].isnull() ]\
                              .drop(['metro_dist', 'g_lift', 'build_tech', 'id'], axis = 1)\
                              .astype(int).values),\
                              axis=0)

                    
                  
y_pre_data = np.concatenate( (train_df.loc[ ~train_df[label_name]\
                                      .isnull() ][label_name].astype(int).values,
                              test_df.loc[ ~test_df[label_name]\
                                     .isnull() ][label_name].astype(int).values),\
                              axis=0)
                              

print('X_pre_train:\nshape: ', X_pre_data.shape, '\ndtype: ', X_pre_data.dtype)
print('y_pre_train:\nshape: ', y_pre_data.shape, '\ndtype: ', y_pre_data.dtype)

X_pre_train:
shape:  (139700, 20) 
dtype:  float64
y_pre_train:
shape:  (139700,) 
dtype:  int64


In [0]:
#begin:
t_before = time.time()

X_train, X_val, y_train, y_val = train_test_split(X_pre_data, y_pre_data, test_size = 0.333, random_state=7)

model = RandomForestClassifier(random_state=7)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)

acc_score = ACC(y_pred, y_val)
f1_score  = F1(y_pred, y_val, average='weighted')

model.fit(X_pre_data, y_pre_data)

t_after = time.time()
#end.

print('acc_score:', "%.3f" % acc_score, '  ',\
      'f1_score:',  "%.3f" % f1_score, '  ',\
      'time:', "%.2f" % (t_after - t_before))

acc_score: 0.502    f1_score: 0.507    time: 4.27


In [0]:
X_pre_test_train = train_df.loc[ train_df[label_name].isnull() ].drop(['metro_dist', 'g_lift', 'build_tech', 'price', 'id'], axis=1).astype("float64").values
y_pre_test_train = model.predict(X_pre_test_train)
train_df.loc[ train_df[label_name].isnull(), label_name ] = y_pre_test_train

In [0]:
X_pre_test = test_df.loc[ test_df[label_name].isnull() ].drop(['metro_dist', 'g_lift', 'build_tech', 'id'], axis=1).astype("float64").values
y_pre_test = model.predict(X_pre_test)
test_df.loc[ test_df[label_name].isnull(), label_name ] = y_pre_test

#***'metro_dist':***

In [0]:
label_name = 'metro_dist'
X_pre_data = np.concatenate( (train_df.loc[ ~train_df[label_name].isnull() ]\
                              .drop(['metro_dist', 'g_lift', 'build_tech', 'price', 'id'], axis = 1)\
                              .astype("float64").values,\
                              test_df.loc[ ~test_df[label_name].isnull() ]\
                              .drop(['metro_dist', 'g_lift', 'build_tech', 'id'], axis = 1)\
                              .astype(int).values),\
                              axis=0)

                    
                  
y_pre_data = np.concatenate( (train_df.loc[ ~train_df[label_name]\
                                      .isnull() ][label_name].astype(int).values,
                              test_df.loc[ ~test_df[label_name]\
                                     .isnull() ][label_name].astype(int).values),\
                              axis=0)
                              

print('X_pre_train:\nshape: ', X_pre_data.shape, '\ndtype: ', X_pre_data.dtype)
print('y_pre_train:\nshape: ', y_pre_data.shape, '\ndtype: ', y_pre_data.dtype)

X_pre_train:
shape:  (190048, 20) 
dtype:  float64
y_pre_train:
shape:  (190048,) 
dtype:  int64


In [0]:
#begin:
t_before = time.time()

X_train, X_val, y_train, y_val = train_test_split(X_pre_data, y_pre_data, test_size = 0.333, random_state=7)

model = RandomForestClassifier(random_state=7)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)

acc_score = ACC(y_pred, y_val)
f1_score  = F1(y_pred, y_val, average='weighted')

model.fit(X_pre_data, y_pre_data)

t_after = time.time()
#end.

print('acc_score:', "%.3f" % acc_score, '  ',\
      'f1_score:',  "%.3f" % f1_score, '  ',\
      'time:', "%.2f" % (t_after - t_before))

acc_score: 0.281    f1_score: 0.302    time: 6.67


In [0]:
X_pre_test_train = train_df.loc[ train_df[label_name].isnull() ].drop(['metro_dist', 'g_lift', 'build_tech', 'price', 'id'], axis=1).astype("float64").values
y_pre_test_train = model.predict(X_pre_test_train)
train_df.loc[ train_df[label_name].isnull(), label_name ] = y_pre_test_train

In [0]:
X_pre_test = test_df.loc[ test_df[label_name].isnull() ].drop(['metro_dist', 'g_lift', 'build_tech', 'id'], axis=1).astype("float64").values
y_pre_test = model.predict(X_pre_test)
test_df.loc[ test_df[label_name].isnull(), label_name ] = y_pre_test

#***Сохраняем обновленные датасеты:***

In [0]:
train_df.to_csv("Train_plus.csv", index=True)
test_df.to_csv("Test_plus.csv", index=True)

from google.colab import files
files.download('Train_plus.csv')
files.download('Test_plus.csv')

#***Загружаем обновленные датасеты:***

In [0]:
import numpy as np
import pandas as pd

train_df = pd.read_csv('Train_plus.csv').drop(['Unnamed: 0'], axis=1)
test_df  = pd.read_csv('Test_plus.csv').drop(['Unnamed: 0'], axis=1)

train_df.head()

,id,street_id,build_tech,floor,area,rooms,balcon,metro_dist,g_lift,n_photos,kw1,kw2,kw3,kw4,kw5,kw6,kw7,kw8,kw9,kw10,kw11,kw12,kw13,price,time
0,0,531,1.0,12,35,1,1,25.0,0.0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,3721000,129384000.0
1,1,385,2.0,15,46,1,2,15.0,1.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,7431000,129384000.0
2,2,51,0.0,1,53,3,0,30.0,0.0,7,0,1,0,0,0,0,0,0,0,0,0,0,0,3081000,129384000.0
3,3,76,0.0,1,60,3,0,30.0,0.0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,3847000,129384000.0
4,4,114,0.0,2,43,2,0,30.0,0.0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,2323000,129384000.0


In [0]:
X_data = train_df.drop(['price', 'id'], axis = 1).astype("float64").values
y_data = train_df['price'].astype("float64").values
X_test = test_df.drop(['id'], axis = 1).astype("float64").values

print('X_train:\nshape: ', X_data.shape, '\ndtype: ', X_data.dtype)
print('y_train:\nshape: ', y_data.shape, '\ndtype: ', y_data.dtype)
print('X_test:\nshape: ', X_test.shape, '\ndtype: ', X_test.dtype)

X_train:
shape:  (100000, 23) 
dtype:  float64
y_train:
shape:  (100000,) 
dtype:  float64
X_test:
shape:  (100000, 23) 
dtype:  float64


#***Features инженеринг:*** (игнорировать)

In [0]:
for i in range(1,13):
  for j in range(i+1,14):
    proc_train_df['kw'+str(i)+'_'+str(j)] = proc_train_df['kw'+str(i)] * proc_train_df['kw'+str(j)]
    proc_test_df['kw'+str(i)+'_'+str(j)]  = proc_test_df['kw'+str(i)]  * proc_test_df['kw'+str(j)]

In [0]:
X_data = proc_train_df.drop(['price', 'id'], axis = 1).astype("float64").values
y_data = proc_train_df['price'].astype("float64").values
X_test = proc_test_df.drop(['id'], axis = 1).astype("float64").values

print('X_train:\nshape: ', X_data.shape, '\ndtype: ', X_data.dtype)
print('y_train:\nshape: ', y_data.shape, '\ndtype: ', y_data.dtype)
print('X_test:\nshape: ', X_test.shape, '\ndtype: ', X_test.dtype)

X_train:
shape:  (100000, 101) 
dtype:  float64
y_train:
shape:  (100000,) 
dtype:  float64
X_test:
shape:  (100000, 101) 
dtype:  float64


In [0]:
#MAE: 11.3953 *23
#MAE: 11.5778 *101

#begin:
t_before = time.time()
X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size = 0.333, random_state=7)

model = GradientBoostingRegressor(learning_rate=0.9)

model.fit(X_train, y_train)
y_pred = model.predict(X_val)

mae_score = MAE(y_pred, y_val)
t_after = time.time()
#end.

print('MAE_score = ', "%.4f" % (mae_score / 100000),\
      '  time:', "%.2f" % (t_after-t_before),'s')

MAE_score =  11.5778   time: 38.03 s


In [0]:
#MAE: 11.3953 *23
#MAE: 11.5778 *101

#score_history_without_columns = pd.DataFrame()

for epoch in range(70):
  min_mae = 999999999.999
  num_col = 0
  for column in range(X_data.shape[1]):
      #begin:
      t_before = time.time()
      X_data_tmp = np.delete(X_data, column, 1)
      X_train, X_val, y_train, y_val = train_test_split(X_data_tmp, y_data, test_size = 0.333, random_state=7)
      y_data_tmp = y_data

      model = GradientBoostingRegressor(learning_rate=0.9)
      
      model.fit(X_train, y_train)
      y_pred = model.predict(X_val)
      
      mae_score = MAE(y_pred, y_val)
      t_after = time.time()
      #end.

      if mae_score < min_mae:
        min_mae = mae_score
        num_col = column

      print('MAE_cross = ', "%.4f" % (mae_score / 100000),\
            '  column:', column, '  X.shape:', X_data.shape[1],\
            '  min_mae:', "%.4f" % (min_mae / 100000), '  num_col:', num_col,\
            '  time:', "%.2f" % (t_after-t_before),'s')

  X_data = np.delete(X_data, num_col, 1)
    
  tmp_df = pd.DataFrame({'MAE_score': [min_mae / 100000], 'column': [num_col]})
  score_history_without_columns = score_history_without_columns.append(tmp_df)

In [0]:
score_history_without_columns.to_csv("score_history_without_columns_(GB).csv", index=True)

from google.colab import files
files.download('score_history_without_columns_(GB).csv')

In [0]:
score_history_without_columns

,MAE_score,column
0,11.392075,8
0,11.117040,2
0,11.074891,16
0,11.072250,64


In [0]:
#MAE: 11.3953 *23
#MAE: 11.5787 *101

score_history_with_usage_func = pd.DataFrame()

for column in range(X_data.shape[1]):
    #begin:
    t_before = time.time()
    X_data_tmp = X_data.copy()
    X_data_tmp[:,column] = (X_data_tmp[:,column]**(1/2))
    X_train, X_val, y_train, y_val = train_test_split(X_data_tmp, y_data, test_size = 0.333, random_state=7)

    model = GradientBoostingRegressor(learning_rate=0.9)
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    
    #mae_score = cross_val_score(model, X_data_tmp, y_data, cv=5, scoring=make_scorer(MAE)).mean()
    mae_score = MAE(y_pred, y_val)
    t_after = time.time()
    #end.

    print('MAE_score = ', "%.4f" % (mae_score / 100000),\
          '  time:', "%.2f" % (t_after-t_before),'s', '   №_col:', column)
    
    tmp_df = pd.DataFrame({'MAE_score': [mae_score], 'column': [column]})
    score_history_with_usage_func = score_history_without_columns.append(tmp_df)

In [0]:
print('X_test:\n', X_test.shape)

for column in score_history_without_columns.column:
  X_test = np.delete(X_test, column, 1)

print(X_test.shape)

X_test:
 (100000, 101)
(100000, 96)


In [0]:
score_history_without_columns.to_csv("score_history_without_columns_(GB).csv", index=True)

from google.colab import files
files.download('score_history_without_columns_(GB).csv')

#***Обучение и подбор параметров модели:*** (игнорировать)

In [0]:
#begin:
t_before_fit = time.time()

model = GradientBoostingRegressor(learning_rate=0.9)
model.fit(X_train, y_train)

y_pred = model.predict(X_val)

mae_score = MAE(y_pred, y_val)

#mae_cross_val_score = cross_val_score(model, X_data, y_data, cv=5, scoring=make_scorer(MAE)).mean()

t_after_fit = time.time()
#end.
      
print('MAE_cvs: ', "%.2f" % (mae_score/100000), '\n',\
      'time:  ', "%.2f" % (t_after_fit - t_before_fit),'s')

MAE_cvs:  11.39 
 time:   11.03 s


In [0]:
#lr=0.12, n_ext=11000: mae=7.27, mae:7.22
#begin:
t_before_fit = time.time()

model = GradientBoostingRegressor(learning_rate=1)
model.fit(X_train, y_train)

y_pred = model.predict(X_val)

mae_score = MAE(y_pred, y_val)

#mae_cross_val_score = cross_val_score(model, X_data, y_data, cv=5, scoring=make_scorer(MAE)).mean()

t_after_fit = time.time()
#end.
      
print('MAE_cvs: ', "%.2f" % (mae_score/100000), '  ',\
      'time:  ', "%.2f" % (t_after_fit - t_before_fit),'s')

MAE_cvs:  11.52    time:   6.00 s


In [0]:
#lr=0.12, n_ext=11000: mae=
X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size = 0.333, random_state=7)

#begin:
t_before_fit = time.time()

lr = 0.9
n_est = 1000

model = GradientBoostingRegressor(learning_rate=lr)#, n_estimators=n_est)
model.fit(X_train, y_train)

y_pred = model.predict(X_val)

mae_score = MAE(y_pred, y_val)

t_after_fit = time.time()
#end.
      
print('n_estimators:', n_est, '  ',\
      'MAE_cvs:', "%.2f" % (mae_score/100000), '  ',\
      'time:', "%.2f" % (t_after_fit - t_before_fit),'s')
#print()

n_estimators: 1000    MAE_cvs: 11.39    time: 7.26 s


In [0]:
#lr=0.12, n_ext=11000: mae=7.27
X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size = 0.333, random_state=7)

for lose_name in ['ls','lad','huber','quantile']:
  
  #begin:
  t_before_fit = time.time()

  lr = 0.5
  n_est = 1000
  #loss_name = i

  model = GradientBoostingRegressor(learning_rate=lr,\
                                    n_estimators=n_est,\
                                    loss=lose_name,
                                    random_state = 7)
  model.fit(X_train, y_train)

  y_pred = model.predict(X_val)

  mae_score = MAE(y_pred, y_val)

  t_after_fit = time.time()
  #end.
        
  print('lose_name:', lose_name, '  ',\
        'MAE_cvs:', "%.2f" % (mae_score/100000), '  ',\
        'time:', "%.2f" % (t_after_fit - t_before_fit),'s')
  #print()

lose_name: ls    MAE_cvs: 7.80    time: 78.73 s
lose_name: lad    MAE_cvs: 8.27    time: 81.93 s
lose_name: huber    MAE_cvs: 7.32    time: 98.95 s
lose_name: quantile    MAE_cvs: 17.09    time: 75.35 s


In [0]:
#lr=0.12, n_ext=11000: mae=7.27
X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size = 0.333, random_state=7)

for lose_name in ['ls','lad','huber','quantile']:
  
  #begin:
  t_before_fit = time.time()

  lr = 0.5
  n_est = 1000
  #loss_name = i

  model = GradientBoostingRegressor(learning_rate=lr,\
                                    n_estimators=n_est,\
                                    loss=lose_name,
                                    random_state = 7)
  model.fit(X_train, y_train)

  y_pred = model.predict(X_val)

  mae_score = MAE(y_pred, y_val)

  t_after_fit = time.time()
  #end.
        
  print('lose_name:', lose_name, '  ',\
        'MAE_cvs:', "%.2f" % (mae_score/100000), '  ',\
        'time:', "%.2f" % (t_after_fit - t_before_fit),'s')
  #print()

lose_name: ls    MAE_cvs: 7.67    time: 57.41 s
lose_name: lad    MAE_cvs: 8.15    time: 67.65 s
lose_name: huber    MAE_cvs: 7.20    time: 83.76 s
lose_name: quantile    MAE_cvs: 15.87    time: 61.72 s


In [0]:
#X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size = 0.333, random_state=7)

history_score = pd.DataFrame()

n_est = 4000
for i in range(10,-1,-1):


  left = 0.23
  right = 0.32
  lr = (1-i/10)*(right-left) + left

  #begin:
  t_before_fit = time.time()


  model = GradientBoostingRegressor(learning_rate=lr,\
                                    n_estimators=n_est,\
                                    loss='huber',
                                    random_state=7)
  model.fit(X_train, y_train)

  y_pred = model.predict(X_val)

  mae_score = MAE(y_pred, y_val)

  t_after_fit = time.time()
  #end.

  tmp_df = pd.DataFrame({'MAE': [mae_score],\
                          'learning_rate': [lr],\
                          'n_estimators': [n_est]})
  
  history_score = history_score.append(tmp_df)

  print('n_est:', n_est, '   ',\
        'lr:', "%.2f" % lr, '   ',\
        'MAE_cvs:', "%.2f" % (mae_score/100000), '   ',\
        'time:', "%.2f" % (t_after_fit - t_before_fit),'s')

n_est: 4000     lr: 0.23     MAE_cvs: 6.90     time: 395.20 s
n_est: 4000     lr: 0.24     MAE_cvs: 6.92     time: 377.99 s
n_est: 4000     lr: 0.25     MAE_cvs: 7.03     time: 387.21 s
n_est: 4000     lr: 0.26     MAE_cvs: 7.26     time: 411.21 s
n_est: 4000     lr: 0.27     MAE_cvs: 7.04     time: 408.31 s


KeyboardInterrupt: ignored

In [0]:
X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size = 0.333, random_state=7)

history_score = pd.DataFrame()

n_est = 6000
for i in range(40,-1,-1):


  left = 0.001
  right = 0.4
  lr = (1-i/40)*(right-left) + left

  #begin:
  t_before_fit = time.time()


  model = GradientBoostingRegressor(learning_rate=lr,\
                                    n_estimators=n_est,\
                                    loss='huber',
                                    random_state=7)
  model.fit(X_train, y_train)

  y_pred = model.predict(X_val)

  mae_score = MAE(y_pred, y_val)
  t_after_fit = time.time()
  #end.

  tmp_df = pd.DataFrame({'MAE': [mae_score],\
                         'learning_rate': [lr],\
                         'n_estimators': [n_est]})
  
  history_score = history_score.append(tmp_df)

  print('n_est:', n_est, '   ',\
        'lr:', "%.4f" % lr, '   ',\
        'MAE_score:', "%.2f" % (mae_score/100000), '   ',\
        'time:', "%.2f" % (t_after_fit - t_before_fit),'s')

n_est: 6000     lr: 0.0010     MAE_score: 16.56     time: 562.55 s
n_est: 6000     lr: 0.0110     MAE_score: 12.45     time: 526.20 s
n_est: 6000     lr: 0.0210     MAE_score: 10.78     time: 545.16 s
n_est: 6000     lr: 0.0309     MAE_score: 9.70     time: 541.68 s
n_est: 6000     lr: 0.0409     MAE_score: 8.98     time: 538.19 s
n_est: 6000     lr: 0.0509     MAE_score: 8.43     time: 535.31 s
n_est: 6000     lr: 0.0609     MAE_score: 7.99     time: 531.91 s
n_est: 6000     lr: 0.0708     MAE_score: 7.84     time: 537.13 s
n_est: 6000     lr: 0.0808     MAE_score: 7.50     time: 523.92 s
n_est: 6000     lr: 0.0908     MAE_score: 7.53     time: 528.00 s
n_est: 6000     lr: 0.1008     MAE_score: 7.78     time: 549.76 s
n_est: 6000     lr: 0.1107     MAE_score: 7.52     time: 552.27 s
n_est: 6000     lr: 0.1207     MAE_score: 7.19     time: 536.90 s
n_est: 6000     lr: 0.1307     MAE_score: 7.30     time: 549.09 s
n_est: 6000     lr: 0.1406     MAE_score: 7.03     time: 545.77 s
n_est: 

In [0]:
X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size = 0.333, random_state=7)

n_est = 6000
lr = 0.31666

#begin:
t_before_fit = time.time()


model = GradientBoostingRegressor(learning_rate=lr,\
                                  n_estimators=n_est,\
                                  loss='huber',
                                  random_state=7)
model.fit(X_train, y_train)

y_pred = model.predict(X_val)

mae_score = MAE(y_pred, y_val)
t_after_fit = time.time()
#end.

tmp_df = pd.DataFrame({'MAE': [mae_score],\
                        'learning_rate': [lr],\
                        'n_estimators': [n_est]})

print('n_est:', n_est, '   ',\
      'lr:', "%.4f" % lr, '   ',\
      'MAE_score:', "%.2f" % (mae_score/100000), '   ',\
      'time:', "%.2f" % (t_after_fit - t_before_fit),'s')

In [0]:
print('n_est:', n_est, '   ',\
      'lr:', "%.4f" % lr, '   ',\
      'MAE_score:', "%.2f" % (mae_score/100000), '   ',\
      'time:', "%.2f" % (t_after_fit - t_before_fit),'s')

n_est: 6000     lr: 0.3167     MAE_score: 6.95     time: 529.95 s


In [0]:
history_score

,MAE,learning_rate,n_estimators
0,1.656151e+06,0.001000,6000
0,1.244522e+06,0.010975,6000
0,1.078390e+06,0.020950,6000
0,9.695196e+05,0.030925,6000
0,8.981038e+05,0.040900,6000
0,8.432966e+05,0.050875,6000
0,7.987409e+05,0.060850,6000
0,7.839749e+05,0.070825,6000
0,7.502300e+05,0.080800,6000
0,7.526575e+05,0.090775,6000


In [42]:
X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size = 0.333, random_state=7)

n_est=11000
for i in range(2,6):

  lr = (1-i/5)*(0.20-0.08) + 0.08

  #begin:
  t_before_fit = time.time()

  model = GradientBoostingRegressor(learning_rate=lr,\
                                    n_estimators=n_est,\
                                    loss='huber',
                                    random_state=7)
  model.fit(X_train, y_train)
  y_pred = model.predict(X_val)

  mae_score = MAE(y_pred, y_val)
  mse_score = MSE(y_pred, y_val)
  evs_score = EVS(y_pred, y_val)

  #mae_score = cross_val_score(model, X_data, y_data, cv=5, scoring=make_scorer(MAE)).mean()

  t_after_fit = time.time()
  #end.

  print('n_est:', n_est, '   ',\
        'lr:', "%.5f" % lr, '   ',\
        'MAE_score:', "%.2f" % (mae_score/100000), '   ',\
        'MSE_score:', "%.2f" % (mse_score/100000000), '   ',\
        'EVS_score:', "%.2f" % evs_score, '   ',\
        'time:', "%.2f" % (t_after_fit - t_before_fit),'s')

n_est: 11000     lr: 0.15200     MAE_score: 6.87     MSE_score: 19996.17     EVS_score: 0.86     time: 972.28 s
n_est: 11000     lr: 0.12800     MAE_score: 6.85     MSE_score: 19877.75     EVS_score: 0.86     time: 967.78 s


KeyboardInterrupt: ignored

n_est: 11000     lr: 0.20000     MAE_score: 6.94     MSE_score: 20073.51     EVS_score: 0.86     time: 956.73 s
n_est: 11000     lr: 0.17600     MAE_score: 6.55     MSE_score: 19656.36     EVS_score: 0.87     time: 976.05 s
n_est: 11000     lr: 0.15200     MAE_score: 6.87     MSE_score: 19996.17     EVS_score: 0.86     time: 977.69 s
n_est: 11000     lr: 0.12800     MAE_score: 6.85     MSE_score: 19877.75     EVS_score: 0.86     time: 970.13 s
n_est: 11000     lr: 0.10400     MAE_score: 6.91     MSE_score: 20227.16     EVS_score: 0.85     time: 956.84 s
n_est: 11000     lr: 0.08000     MAE_score: 7.20     MSE_score: 20712.59     EVS_score: 0.85     time: 955.08 s


In [0]:
history_score

NameError: ignored

In [0]:
#n_est=11000, lr=0.12, time=769s: mae=7.17
#n_est=4000,  lr=0.3,  time=401s: mae=7.28

#***Формирование .csv файла:***

In [43]:
#lr = 0.31, n_est = 6000
#lr = 0.12, n_est = 11000

#begin:
t_before_fit = time.time()

model = GradientBoostingRegressor(learning_rate=0.176,\
                                  n_estimators=11000,\
                                  loss='huber',
                                  random_state=7)

model.fit(X_data, y_data)

t_after_fit = time.time()
#end.

print('time:  ', "%.2f" % (t_after_fit - t_before_fit),'s')

time:   1265.48 s


In [44]:
#begin:
t_before_fit = time.time()

y_pred = model.predict(X_test)
test_df['price'] = y_pred

t_after_fit = time.time()
#end.

print('time:  ', "%.2f" % (t_after_fit - t_before_fit),'s')

time:   18.04 s


In [0]:
test_df[["id", "price"]].to_csv("submission1.csv", index=False)

from google.colab import files
files.download('submission1.csv')